# Functions and imports

In [1]:
import time
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
import os
import re#
import pandas as pd
import numpy as np
from IPython.display import Audio
import seaborn as sns

import helpers # this is where the main training/decoding functions are, modified from teh original HIVAE main.py

#import warnings 
#warnings.filterwarnings('ignore') ########## NOTE: comment out for testing in case it's hiding problems

def set_settings(opts,nepochs=500,modload=False,save=True): # note: modload doesnt do anything right now, hardcoded in helpers.py
    'replace setting template placeholders with file info'
    inputf=re.sub('.csv','',opts['files'].iloc[0])
    #missf=inputf+'_missing.csv'
    typef=inputf+'_types.csv'
    
    template = '--epochs NEPOCHS --model_name model_HIVAE_inputDropout --restore MODLOAD \
        --data_file data_python_Notmissing/INPUT_FILE.csv --types_file data_python_Notmissing/TYPES_FILE \
         --batch_size NBATCH --save NEPFILL --save_file SAVE_FILE\
        --dim_latent_s SDIM --dim_latent_z 1 --dim_latent_y YDIM \
        --learning_rate LRATE'
    
    # replace placeholders in template
    settings = re.sub('INPUT_FILE',inputf,template)
    settings = re.sub('NBATCH',str(opts['nbatch'].iloc[0]),settings)
    settings = re.sub('NEPOCHS',str(nepochs),settings)
    settings = re.sub('NEPFILL',str(nepochs-1),settings) if save else re.sub('NEPFILL',str(nepochs*2),settings)
    settings = re.sub('YDIM',str(opts['ydims'].iloc[0]),settings)
    settings = re.sub('SDIM',str(opts['sdims'].iloc[0]),settings)
    #settings = re.sub('MISS_FILE',missf,settings) #if not 'medhist' in inputf else re.sub('--true_miss_file data_python/MISS_FILE','',settings)
    settings = re.sub('TYPES_FILE',typef,settings)
    settings = re.sub('SAVE_FILE',inputf,settings)
    settings = re.sub('LRATE',str(opts['lrates'].iloc[0]),settings)
    settings = re.sub('MODLOAD','1',settings) if modload else re.sub('MODLOAD','0',settings)
    
    return settings

In [2]:
# get file list
files=[i for i in os.listdir('data_python_Notmissing/') if not '_type' in i and not '_missing' in i and not '.DS_Store' in i and 'stalone' not in i]
files.sort()
print(len(files))

5


In [3]:
file = pd.read_csv('data_python_Notmissing/' + files[1], header=None)
sample_size=len(file)

# General settings

In [4]:
best_hyper = pd.read_csv('best_hyper_ADNI_processed_nm.csv', sep = ",")

In [5]:
best_hyper

,loss,lrates,nbatch,wdecay,ydims,files,sdims
0,0.575540,0.001,32,0.000,1,MMSE_Attention_Concentration_VIS1.csv,1
1,0.822020,0.001,32,0.000,1,MMSE_Language_VIS1.csv,2
2,1.229906,0.010,16,0.000,1,MMSE_Memory_Recall_VIS1.csv,2
3,1.285093,0.001,32,0.000,1,MMSE_Orientation_VIS1.csv,2
4,0.070693,0.001,16,0.001,1,MMSE_Working_Memory_Registration_VIS1.csv,1


# VP decoding

Run after bnet.R

In [6]:
os.getcwd()

'/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper_final/data/HI-VAE'

In [7]:
meta = pd.read_csv('metaenc_nm.csv')
meta


,scode_MMSE_Attention_Concentration_VIS1,SUBJID,zcode_MMSE_Attention_Concentration_VIS1,scode_MMSE_Language_VIS1,zcode_MMSE_Language_VIS1,scode_MMSE_Memory_Recall_VIS1,zcode_MMSE_Memory_Recall_VIS1,scode_MMSE_Orientation_VIS1,zcode_MMSE_Orientation_VIS1,scode_MMSE_Working_Memory_Registration_VIS1,zcode_MMSE_Working_Memory_Registration_VIS1
0,0,002_S_0295,-3.382766,0,-0.698505,0,-16.990568,1,8.774607,0,3.631883
1,0,002_S_0413,-3.382766,0,-1.948777,0,-16.990568,1,8.774607,0,3.631883
2,0,002_S_0559,-3.382766,1,-10.335737,0,-16.990568,1,8.774607,0,3.631883
3,0,002_S_0619,-3.382766,1,-10.335737,1,2.121368,0,-0.696185,0,3.631883
4,0,002_S_0685,-3.382766,1,-10.335737,0,-16.990568,1,8.774607,0,3.631883
...,...,...,...,...,...,...,...,...,...,...,...
1440,0,941_S_4377,-3.382766,0,-1.948777,0,-16.990568,1,8.774607,0,3.631883
1441,0,941_S_4420,-2.850877,0,-1.948777,0,-16.990568,1,8.774607,0,3.631883
1442,0,941_S_4764,-3.382766,1,-10.335737,0,-16.990568,1,8.774607,0,3.631883
1443,0,941_S_5124,-3.382766,0,-1.948777,1,1.046212,0,0.418486,0,3.631883


In [8]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper_final/data/data_out/main_VirtualPPts_nm.csv')
VPcodes

,zcode_MMSE_Attention_Concentration_VIS1,scode_MMSE_Language_VIS1,zcode_MMSE_Language_VIS1,scode_MMSE_Memory_Recall_VIS1,zcode_MMSE_Memory_Recall_VIS1,scode_MMSE_Orientation_VIS1,zcode_MMSE_Orientation_VIS1,zcode_MMSE_Working_Memory_Registration_VIS1
0,-3.166412,1,-10.335737,0,-16.990568,0,1.311595,3.575307
1,-3.624268,1,-10.335737,1,1.699833,1,8.774607,3.784365
2,-3.274005,0,-1.841420,1,1.423218,1,8.774607,3.733536
3,-3.224661,1,-10.335737,0,-16.990568,1,8.774607,3.555333
4,-3.047882,1,-10.335737,0,-16.990568,0,0.962834,3.628081
...,...,...,...,...,...,...,...,...
1440,-3.539807,0,-1.356281,1,1.600307,0,1.710450,3.555597
1441,-2.890982,1,-10.335737,1,1.868000,0,0.648495,3.652027
1442,-3.498688,0,-1.963121,1,1.981911,0,0.256513,3.699737
1443,-3.390105,0,-2.170953,1,1.977013,0,0.438483,3.893234


In [9]:
VPcodes.columns

Index(['zcode_MMSE_Attention_Concentration_VIS1', 'scode_MMSE_Language_VIS1',
       'zcode_MMSE_Language_VIS1', 'scode_MMSE_Memory_Recall_VIS1',
       'zcode_MMSE_Memory_Recall_VIS1', 'scode_MMSE_Orientation_VIS1',
       'zcode_MMSE_Orientation_VIS1',
       'zcode_MMSE_Working_Memory_Registration_VIS1'],
      dtype='object')

In [10]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper_final/data/data_out/main_VirtualPPts_nm.csv')

dfs=list()
virt=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    dec=helpers.dec_network(settings,zcodes,scodes,VP='nomiss')
    subj=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_subj.csv')['x']
    names=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_cols.csv')['x']
    dat=pd.DataFrame(dec)
    dat.columns=names
    dat['SUBJID']=subj
    virt.append(dec)
    dfs.append(dat)

virt_dic=dict(zip(files,virt))
decoded=helpers.merge_dat(dfs)
decoded.to_csv('decodedVP_nm.csv',index=False)

VP
nomiss
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders

[*] Defining Encoder...
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.



[*] Defining Decoder...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use `tf.random.categorical` instead.
[*] Defining Cost function...
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Attention_Concentration_VIS1/MMSE_Attention_Concentration_VIS1.ckpt
Model restore

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Language_VIS1/MMSE_Language_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Language_VIS1/MMSE_Language_VIS1.ckpt
::::::DECODING:::::::::
feed Dictionary
{<tf.Tensor 'Placeholder:0' shape=(1445, 6) dtype=float32>: array([[1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1.]])}
:::::::::::: ones for miss list VP
Number of cols
1
minimum
4
maximum
9
VP
nomiss
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...


...

[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Memory_Recall_VIS1/MMSE_Memory_Recall_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Memory_Recall_VIS1/MMSE_Memory_Recall_VIS1.ckpt
::::::DECODING:::::::::
feed Dictionary
{<tf.Tensor 'Placeholder:0' shape=(1445, 4) dtype=float32>: array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])}
:::::::::::: ones for miss list VP
Number of cols
1
minimum
0
maximum
3
VP
nomiss
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Orientation_VIS1/MMSE_Orientation_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Orientation_VIS1/MMSE_Orientation_VIS1.ckpt
::::::DECODING:::::::::
feed Dictionary
{<tf.Tensor 'Placeholder:0' shape=(1445, 7) dtype=float32>: array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 1., 1., 0.]])}
:::::::::::: ones for miss list VP
Number of cols
1
minimum
4
maximum
10
VP
nomiss
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Working_Memory_Registration_VIS1/MMSE_Working_Memory_Registration_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Working_Memory_Registration_VIS1/MMSE_Working_Memory_Registration_VIS1.ckpt
::::::DECODING:::::::::
feed Dictionary
{<tf.Tensor 'Placeholder:0' shape=(1445, 3) dtype=float32>: array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       ...,
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])}
:::::::::::: ones for miss list VP
Number of cols
1
minimum
1
maximum
3


Get Loglikelihoods for R plot!

In [11]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper/data/data_out/main_VirtualPPts_nm.csv')

dfs=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    loglik=helpers.dec_network_loglik(settings,zcodes,scodes,VP='nomiss')
    loglik=np.nanmean(np.array(loglik).T,axis=1)
    subj=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_subj.csv')['x']
    dat=pd.DataFrame(loglik)
    dat.columns=[f]
    dat['SUBJID']=subj
    dfs.append(dat)

decoded=helpers.merge_dat(dfs)
decoded.to_csv('virtual_logliks_nm.csv',index=False)

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=1445, data_file='data_python_Notmissing/MMSE_Attention_Concentration_VIS1.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.001, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='MMSE_Attention_Concentration_VIS1', true_miss_file=None, types_file='data_python_Notmissing/MMSE_Attention_Concentration_VIS1_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Attention_Concentration_VIS1/MMSE_Attention_Concentration_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Attention_Concentration_VIS1/MMSE_Attention_Concentration_VIS1.ckpt
::::::DECODING:::::::::
:::::::::::: ones for miss list VP
Namespace(batch_size=1445,

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Language_VIS1/MMSE_Language_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Language_VIS1/MMSE_Language_VIS1.ckpt
::::::DECODING:::::::::
:::::::::::: ones for miss list VP
Namespace(batch_size=1445, data_file='data_python_Notmissing/MMSE_Memory_Recall_VIS1.csv', dim_latent_s=2, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='MMSE_Memory_Recall_VIS1', true_miss_file=None, types_file='data_python_Notmissing/MMSE_Memory_Recall_VIS1_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Memory_Recall_VIS1/MMSE_Memory_Recall_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Memory_Recall_VIS1/MMSE_Memory_Recall_VIS1.ckpt
::::::DECODING:::::::::
:::::::::::: ones for miss list VP
Namespace(batch_size=1445, data_file='data_python_Notmissing/MMSE_Orientation_VIS1.csv', dim_latent_s=2, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.001, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='MMSE_Orientation_VIS1', true_miss_file=None, types_file='data_python_Notmissing/MMSE_Orientation_VIS1_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Orientation_VIS1/MMSE_Orientation_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Orientation_VIS1/MMSE_Orientation_VIS1.ckpt
::::::DECODING:::::::::
:::::::::::: ones for miss list VP
Namespace(batch_size=1445, data_file='data_python_Notmissing/MMSE_Working_Memory_Registration_VIS1.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.001, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='MMSE_Working_Memory_Registration_VIS1', true_miss_file=None, types_file='data_python_Notmissing/MMSE_Working_Memory_Registration_VIS1_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/MMSE_Working_Memory_Registration_VIS1/MMSE_Working_Memory_Registration_VIS1.ckpt
Model restored: ./Saved_Networks/MMSE_Working_Memory_Registration_VIS1/MMSE_Working_Memory_Registration_VIS1.ckpt
::::::DECODING:::::::::
:::::::::::: ones for miss list VP
